In [ ]:
!pip install cartopy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 81.7 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import  xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
import warnings
import math
from tqdm.auto import tqdm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cartopy.feature as cf
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker

from cartopy.util import add_cyclic_point
from matplotlib import animation
from tqdm.auto import tqdm
from datetime import datetime,timedelta

from matplotlib.cm import get_cmap
warnings.filterwarnings("ignore")

In [ ]:
ds=xr.open_dataset('/content/drive/MyDrive/AP_HW/ERA5 Data/potential_vorticity/pv-2024.nc')
ds

<xarray.Dataset> Size: 3GB
Dimensions:         (valid_time: 2928, pressure_level: 3, latitude: 209,
                     longitude: 381)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 23kB 2024-01-01 ... 2024-12-3...
  * pressure_level  (pressure_level) float64 24B 500.0 300.0 200.0
  * latitude        (latitude) float64 2kB 52.0 51.75 51.5 ... 0.5 0.25 0.0
  * longitude       (longitude) float64 3kB 25.0 25.25 25.5 ... 119.8 120.0
    expver          (valid_time) <U4 47kB ...
Data variables:
    pv              (valid_time, pressure_level, latitude, longitude) float32 3GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-03-26T09:04 GRIB to CDM+CF via cfgrib-0.9.1...

# MAM

In [ ]:
# @title Heat date list
list_hw=pd.read_csv('/content/drive/MyDrive/AP_HW/Scripts-ll/HW_dates/data/HW_list_rolling_def.csv')
list_hw
list_hw['Starting_Date'] = pd.to_datetime(list_hw["Starting_Date"])
list_hw['Ending_Date'] = pd.to_datetime(list_hw["Ending_Date"])

# separate hw for mam season
mam_hw_list =list_hw[(list_hw['Starting_Date'].dt.month >= 3) & (list_hw['Starting_Date'].dt.month <= 5)]
mam_hw_list.reset_index(inplace=True,drop=True)
mam_hw_list
# list_hw

,Starting_Date,Ending_Date,Duration
0,1972-04-23,1972-04-25,3
1,1972-05-08,1972-05-19,12
2,1973-03-31,1973-04-05,6
3,1973-04-07,1973-04-10,4
4,1973-04-15,1973-04-19,5
...,...,...,...
99,2023-05-29,2023-06-08,11
100,2024-04-01,2024-04-06,6
101,2024-04-12,2024-05-05,24
102,2024-05-14,2024-05-17,4


In [ ]:
#@title Event data for composite analysis-vertically avareged
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import matplotlib.colors as colors

pv_anom_vertical_list=[]

for i in tqdm(range(len(mam_hw_list['Starting_Date']))):
  year=mam_hw_list['Starting_Date'].dt.year.iloc[i]
  start_time =str(mam_hw_list['Starting_Date'][i])
  end_time = str(mam_hw_list['Ending_Date'][i])
  start_time =str(np.datetime64(start_time) - np.timedelta64(6, 'h')) # Substracting 6hrs to convert to BD time.
  end_time =str(np.datetime64(end_time) - np.timedelta64(6, 'h')+ np.timedelta64(24,'h')) # Adding 24hrs to include the end date too.
  time_range=slice(start_time,end_time)
  event_no=i+1
  ###################################
  ############# pblh ################
  # Load the files
  ds_pv = xr.open_dataset(f'/content/drive/MyDrive/AP_HW/ERA5 Data/potential_vorticity/pv-{year}.nc')
  pv_clim=xr.open_dataset('/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/pv/pv_mam_vertical_climatology.nc')


  ds_pv=ds_pv.sel(valid_time=time_range)
  ds_pv=ds_pv.mean(dim='pressure_level',skipna=True)
  ds_pv=ds_pv.mean(dim='valid_time',skipna=True)
  pv_anom=ds_pv['pv']-pv_clim['pv']
  pv_anom=pv_anom.assign_coords(event_no=event_no)
  pv_anom=pv_anom.to_dataset()
  pv_anom_vertical_list.append(pv_anom)

pv_events_anom_mam_level=xr.concat(pv_anom_vertical_list, dim='event_no')
pv_events_anom_mam_level.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/pv/data/pv_level_avg_anom-mam.nc')


  0%|          | 0/104 [00:00<?, ?it/s]

# JJAS

In [ ]:
# @title Heat date list
list_hw=pd.read_csv('/content/drive/MyDrive/AP_HW/Scripts-ll/HW_dates/data/HW_list_rolling_def.csv')
list_hw['Starting_Date'] = pd.to_datetime(list_hw["Starting_Date"])
list_hw['Ending_Date'] = pd.to_datetime(list_hw["Ending_Date"])

# separate hw for jjas season
jjas_hw_list =list_hw[(list_hw['Starting_Date'].dt.month >= 6) & (list_hw['Starting_Date'].dt.month <= 9)]
jjas_hw_list.reset_index(inplace=True,drop=True)

In [ ]:
#@title Event data for composite analysis-vertically avareged

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import matplotlib.colors as colors

pv_anom_vertical_list=[]

for i in tqdm(range(len(jjas_hw_list['Starting_Date']))):
  year=jjas_hw_list['Starting_Date'].dt.year.iloc[i]
  start_time =str(jjas_hw_list['Starting_Date'][i])
  end_time = str(jjas_hw_list['Ending_Date'][i])
  start_time =str(np.datetime64(start_time) - np.timedelta64(6, 'h')) # Substracting 6hrs to convert to BD time.
  end_time =str(np.datetime64(end_time) - np.timedelta64(6, 'h')+ np.timedelta64(24,'h')) # Adding 24hrs to include the end date too.
  time_range=slice(start_time,end_time)
  event_no=i+1
  ###################################
  ############# pblh ################
  # Load the files
  ds_pv = xr.open_dataset(f'/content/drive/MyDrive/AP_HW/ERA5 Data/potential_vorticity/pv-{year}.nc')
  pv_clim=xr.open_dataset('/content/drive/MyDrive/AP_HW/Scripts-ll/climatology/data/pv/pv_jjas_vertical_climatology.nc')

  ds_pv=ds_pv.sel(valid_time=time_range)
  ds_pv=ds_pv.mean(dim='pressure_level',skipna=True)
  ds_pv=ds_pv.mean(dim='valid_time',skipna=True)
  pv_anom=ds_pv['pv']-pv_clim['pv']
  pv_anom=pv_anom.assign_coords(event_no=event_no)
  pv_anom=pv_anom.to_dataset()
  pv_anom_vertical_list.append(pv_anom)

pv_events_anom_jjas_level=xr.concat(pv_anom_vertical_list, dim='event_no')
pv_events_anom_jjas_level.to_netcdf('/content/drive/MyDrive/AP_HW/Scripts-ll/pv/data/pv_level_avg_anom-jjas.nc')



  0%|          | 0/135 [00:00<?, ?it/s]

In [ ]:
print('done')

done
